## 使用Keras 回调函数和TensorBoard 来检查并监控深度学习模型
本节将介绍在训练过程中如何更好地访问并控制模型内部过程的方法。使用`model.fit()`或`model.fit_generator()`在一个大型数据集上启动数十轮的训练，有点类似于扔一架纸飞机，一开始给它一点推力，之后便再也无法控制其飞行轨迹或着陆点。如果想要避免不好的结果（并避免浪费纸飞机），更聪明的做法是不用纸飞机，而是用一架无人机，它可以感知其环境，将数据发回给操纵者，并且能够基于当前状态自主航行。下面要介绍的技术，可以让`model.fit()`的调用从纸飞机变为智能的自主无人机，可以自我反省并动态地采取行动。

### 训练过程中将回调函数作用于模型
最好的防止防止过拟合的方法是，当观测到验证损失不再改善时就停止训练。这可以使用Keras回调函数来实现。**回调函数（callback）** 是在调用fit时传入模型的一个对象（即实现特定方法的类实例），它在训练过程中的不同时间点都会被模型调用。它可以访问关于模型状态与性能的所有可用数据，还可以采取行动：中断训练、保存模型、加载一组不同的权重或改变模型的状态。回调函数的一些用法示例如下所示。
* **模型检查点（model checkpointing）：** 在训练过程中的不同时间点保存模型的当前权重。
* **提前终止（early stopping）：** 如果验证损失不再改善，则中断训练（当然，同时保存在训练过程中得到的最佳模型）。
* 在训练过程中**动态调节**某些参数值：比如优化器的学习率。
* 在训练过程中**记录训练指标和验证指标**，或将模型学到的表示可视化（这些表示也在不断更新）：Keras进度条就是一个回调函数！

下面介绍几个回调函数：

1. **ModelCheckpoint 与EarlyStopping 回调函数**

    如果监控的目标指标在设定的轮数内不再改善，可以用`EarlyStopping`回调函数来中断训练。比如，这个回调函数可以在刚开始过拟合的时候就中断训练，从而避免用更少的轮次重新训练模型。这个回调函数通常与`ModelCheckpoint`结合使用，后者可以在训练过程中持续不断地保存模型（也可以选择只保存目前的最佳模型，即一轮结束后具有最佳性能的模型）。
    
2. **ReduceLROnPlateau 回调函数**

    如果验证损失不再改善，你可以使用这个回调函数来降低学习率。在训练过程中如果出现了**损失平原（loss plateau）**，那么增大或减小学习率都是跳出局部最小值的有效策略。
    
3. **自定义回调函数**
     
    如果需要在训练过程中采取特定行动，而这项行动又没有包含在内置回调函数中，那么可以编写自定义的回调函数。回调函数的实现方式是创建`keras.callbacks.Callback`类的子类。然后可以实现下列方法，它们分别在训练过程中的不同时间点被调用。
    
    on_epoch_begin, on_epoch_end, on_batch_begin, on_batch_end, on_train_begin, on_train_end
    
    这些方法被调用时都有一个logs 参数，这个参数是一个字典，里面包含前一个批量、前一个轮次或前一次训练的信息，即训练指标和验证指标等。此外，回调函数还可以访问下列属性。
    * `self.model`：调用回调函数的模型实例。
    * `self.validation_data`：传入 fit作为验证数据的值。

In [ ]:
# ModelCheckpoint 与EarlyStopping 回调函数
import keras

# 通过fit 的callbacks 参数将回调函数传入模型中，这个参数接收一个回调函数的列表。可以传入任意个数的回调函数
callbacks_list = [
# patience表示如果精度在多于一轮的时间（即两轮）内不再改善，中断训练
keras.callbacks.EarlyStopping(monitor='acc',patience=1), 
# 参数的含义是，如果val_loss 没有改善，那么不需要覆盖模型文件。这就可以始终保存在训练过程中见到的最佳模型
keras.callbacks.ModelCheckpoint(filepath='my_model.h5',monitor='val_loss',save_best_only=True,)
]

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# 注意，由于回调函数要监控验证损失和验证精度，所以在调用fit时需要传入validation_data
model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))


# ReduceLROnPlateau 回调函数

# 如果验证损失在10轮内都没有改善，那么就触发这个回调函数, 触发时将学习率乘以0.1
callbacks_list = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)]
model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

In [ ]:
# 自定义回调函数
# 下面是一个自定义回调函数的简单示例，它可以在每轮结束后将模型每层的激活保存到硬盘（格式为Numpy数组），这个激活是对验证集的第一个样本计算得到的。

import keras
import numpy as np


class ActivationLogger(keras.callbacks.Callback):
'''这是一个自定义回调函数的简单示例，它可以在每轮结束后将模型每层的激活保存到硬盘（格式为Numpy数组），这个激活是对验证集的第一个样本计算得到的'''

    def set_model(self, model):
        # 在训练之前由父模型调用，告诉回调函数是哪个模型在调用它
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        # 模型实例，返回每层的激活
        self.activations_model = keras.models.Model(model.input,ayer_outputs)
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
            
        # 获取验证数据的第一个输入样本
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        f.close()

### TensorBoard 简介：TensorFlow 的可视化框架
本节将介绍TensorBoard，一个内置于TensorFlow中的基于浏览器的可视化工具。TensorBoard的主要用途是，在训练过程中帮助你以可视化的方法监控模型内部发生的一切。如果你监控了除模型最终损失之外的更多信息，那么可以更清楚地了解模型做了什么、没做什么，并且能够更快地取得进展。TensorBoard 具有下列巧妙的功能，都在浏览器中实现。
* 在训练过程中以可视化的方式监控指标
* 将模型架构可视化
* 将激活和梯度的直方图可视化
* 以三维的形式研究嵌入

用一个简单的例子来演示这些功能：在IMDB 情感分析任务上训练一个一维卷积神经网络。只考虑IMDB 词表中的前2000 个单词，这样更易于将词嵌入可视化。

In [10]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

# 每一轮之后记录激活直方图, 每一轮之后记录嵌入数据
callbacks = [keras.callbacks.TensorBoard(log_dir='log_dir', histogram_freq=1, embeddings_freq=1)]

history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks)

17465344/17464789 [==============================] - 48s 3us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-tr

/Users/s33c0f1/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/callbacks/tensorboard_v2.py:102: UserWarning: The TensorBoard callback does not support embeddings display when using TensorFlow 2.0. Embeddings-related arguments are ignored.
  warnings.warn('The TensorBoard callback does not support '
/Users/s33c0f1/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20000 samples, validate on 5000 samples


AttributeError: 'Sequential' object has no attribute '_get_distribution_strategy'

命令行启动TensorBoard 服务器，指示它读取回调函数当前正在写入的日志。

`$ tensorboard --logdir=my_log_dir`

然后可以用浏览器打开`http://localhost:6006`并查看模型的训练过程。
* 除了训练指标和验证指标的实时图表之外，还可以访问 **HISTOGRAMS** 标签页，并查看美观的直方图可视化，直方图中是每层的激活值。

* **EMBEDDINGS** 标签页让你可以查看输入词表中2000个单词的嵌入位置和空间关系，它们都是由第一个Embedding层学到的。因为嵌入空间是128维的，所以TensorBoard会使用选择的降维算法自动将其降至二维或三维，可选的降维算法有`主成分分析（PCA）`和`t-分布随机近邻嵌入（t-SNE）`。在图示点状云中，可以清楚地看到两个簇：正面含义的词和负面含义的词。从可视化图中可以立刻明显地看出，将嵌入与特定目标联合训练得到的模型是完全针对这个特定任务的，这也是为什么使用预训练的通用词嵌入通常不是一个好主意。

* **GRAPHS** 标签页显示的是Keras 模型背后的底层TensorFlow运算图的交互式可视化。可见，图中的内容比之前想象的要多很多。对于刚刚构建的模型，在Keras中定义模型时可能看起来很简单，只是几个基本层的堆叠；但在底层，需要构建相当复杂的图结构来使其生效。其中许多内容都与梯度下降过程有关。所见到的内容与所操作的内容之间存在这种复杂度差异，这正是你选择使用Keras来构建模型、而不是使用原始TensorFlow从头开始定义所有内容的主要动机。Keras让工作流程变得非常简单。

**Note:** Keras还提供了另一种更简洁的方法——`keras.utils.plot_model`函数，它可以将模型绘制为层组成的图，而不是TensorFlow运算组成的图。使用这个函数需要安装Python的pydot库和pydot-ng库，还需要安装graphviz库。